## 세팅

In [43]:
from pypdf import PdfReader
from openai import OpenAI
import os
from dotenv import load_dotenv
import numpy as np

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

from langchain import hub

from langchain.chat_models import ChatOpenAI

from langchain.llms import OpenAI as LangChainOpenAI

from langchain.chains import RetrievalQA

from langchain.tools.retriever import create_retriever_tool

from langchain.agents import initialize_agent, AgentType

from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain

import time

from langchain.agents import Tool

import faiss

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## PDF에서 텍스트를 읽어 청크로 분할

In [29]:
pdf_list = ["data/rag_data/spiderman1.pdf", "data/rag_data/spiderman2.pdf"]
chunk_length = 2048

chunks = []
for i in range(len(pdf_list)):
    reader = PdfReader(pdf_list[i])
    
    for page in reader.pages:
        text_page = page.extract_text()
        chunks.extend([text_page[i: i+chunk_length] for i in range(0, len(text_page), chunk_length)])

In [30]:
# 청크 개수 확인 및 청크 내용 확인

print(len(chunks))
print(chunks[0])

59
최근 변경 최근 토론
어메이징  스파이더맨  실사영화  시리즈의 등장인물
스파이더맨
Spider -Man스파이더맨 ( 어메이징  스파이더맨  실사영화  시리
즈)
최근 수정  시각 : 2024-06-23 22:49:30
  스파이더맨  트릴로지의  스파이더맨에  대한  내용은  스파이더맨 ( 스파이더맨  트릴로지 ) 문서를, 마블  시
네마틱  유니버스의  스파이더맨에  대한  내용은  스파이더맨 ( 마블  시네마틱  유니버스 ) 문서를 참고하십시오 .
스파이더맨  관련  틀
[ 펼치기  · 접기  ]특수 기능
여기에서  검색
편집 요청 토론 역사
분류: 피터 파커어메이징  스파이더맨  실사영화  시리즈 / 등장인물스파이더맨 ( 마블  시네마틱  유니버스 )…
마블시네마틱 유니버 평행세계의 인물파이더맨 웨이 홈 장인물더 보기
29



In [33]:
# PDF에서 텍스트 읽어서 txt로 저장
pdf_list = ["data/rag_data/spiderman1.pdf", "data/rag_data/spiderman2.pdf"]

with open("data/rag_data/spiderman.txt", "w", encoding="utf-8") as f:
    for pdf_path in pdf_list:
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            text_page = page.extract_text()
            f.write(text_page + "\n")

In [34]:
# 전처리하고 다시 읽어와서 청크로 저장하기
chunks = []
chunk_length = 2048
with open("data/rag_data/spiderman.txt", "r", encoding="utf-8") as f:
    text = f.read()
    chunks.extend([text[i:i+chunk_length] for i in range(0, len(text), chunk_length)])

In [35]:
print(len(chunks))
print(chunks[0])

28
어메이징  스파이더맨  실사영화  시리즈의 등장인물 스파이더맨
Spider -Man스파이더맨 ( 어메이징  스파이더맨  실사영화  시리즈)
스파이더맨 : 노  웨이  홈
본명 피터 벤저민  파커
Peter Benjamin Parker
이명 스파이더맨
Spider -Man
피터 3
Peter-Three
어메이징  스파이더맨
Amazing Spider-Man
종족 인간 (강화인간)
국적 미국
성별 남성
가족 관계벤 파커 (숙부)
메이 파커 (숙모)
리처드  파커 (아버지)
메리 파커  (어머니)
등장 영화〈어메이징  스파이더맨〉
〈어메이징  스파이더맨  2〉
〈스파이더맨 : 노  웨이  홈〉
〈스파이더맨 : 어크로스  더  유니버스〉 (카메오)
담당 배우 앤드류  가필드
담당 성우 정재헌
마에노  토모아키
스파이더맨 : 노  웨이  홈  - 어메이징  피터  #3
어메이징  스파이더맨  실사영화  시리즈의 스파이더맨. 담당  배우는  앤드류  가필드. 1편의  아역  배우는  맥스
찰스가  담당했다.
슈트 디자인, 캐릭터성  등에서  상당한  리파인이  가해진  트릴로지와  MCU 스파이더맨에  비해  코믹스  속  어메
이징 스파이더맨 을 최대한  실사적으로  묘사한  것이  특징. 웹  슈터에  독자적인  소프트웨어를  집어넣거나  실
험을 통해  여러  발명을  하는  등, 스파이더맨  코믹스  특유의  SF 감성을  드러내는  장면이  많다. 거기에  더해  유
쾌한 면모  및  가족에  대한  사랑  같은  장점부터, 우유부단한  면모와  불행으로  주변  인물을  잃고  고통받는  등
원작의  피터를  연상케  하는  모습이  많다. 다만  피터의  여러  캐릭터성  중  한  가지에  초점을  맞춘  다른  두  시리
즈에 비하면  얕고  넓게  다룬다는  느낌이  강하며, 이  때문인지  각각  책임감과  성장이라는  테마가  확고한  트릴
로지와  MCU 스파이더맨에  비해  시리즈  전체를  관통하는  테마가  애매한  편이다. 대신  영화

## FAISS에 벡터 저장

FAISS 내장 함수 정리

1. FAISS.from_texts()

- texts: 벡터 저장소에 추가할 텍스트 리스트(List(str))
- embedding: 사용할 임베딩 함수
- metadatas: 메타데이터 리스트(List[dict])
- ids: 문서 ID 리스트(List(str))

2. vectorstore.add_texts(): 텍스트를 임베딩하고 벡터 저장소에 추가
- texts(Iterable[str]): 벡터 저장소에 추가할 텍스트 이터러블

3. vectorstore.save_local(): FAISS 인덱스, 문서 저장소, 인덱스-문서 ID 매핑을 로컬에 저장
- FAISS 인덱스를 별도의 파일로 저장하고, 문서 저장소와 인덱스-문서 ID 매핑을 pickle 형식으로 저장함
- folder_path: 저장할 폴더 경로
- index_name: 저장할 인덱스 파일 이름(기본값: index)

4. FAISS.load_local(): FAISS 인덱스, 문서 저장소, 인덱스 - 문서 ID 매핑을 불러오는 기능
- folder_path: 불러올 파일들이 저장된 경로
- embeddings: 쿼리 생성에 사용할 임베딩 객체

5. vectorstore.docstore._dict: 벡터스토어에 저장된 문서 확인

In [36]:
# 임베딩 모델 쓸 수 있는 거
# - text-embedding-3-small
# - text-embedding-3-large
# - text-embedding-ada-002 (default)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(chunks, embeddings)

In [ ]:
# 다른 인덱스를 사용하기

embedding_vectors = [embeddings.embed_documents(text) for text in chunks]

# 임베딩 벡터를 패딩 하고 넘파이 배열로 변환
lengths = [len(vec) for vec in embedding_vectors]
max_length = max(lengths)

padded_vectors = []
for vec in embedding_vectors:
    padded_vec = np.pad(vec, (0, max_length - len(vec)), "constant", constant_values=0)
    padded_vectors.append(padded_vec)

embedding_vectors_np = np.array(padded_vectors, dtype="float32")

In [56]:
d = embedding_vectors_np.shape[1]

# IVFFlat index 만들기
nlist = 100 # IVF의 클러스터 수, 적절하게 조정해야 함
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

# 인덱스 훈련
index.train(embedding_vectors_np)

# 벡터 추가
index.add(embedding_vectors_np)

# 랭체인 벡터스토어로 저장
vectorstore = FAISS(embedding_vectors_np, chunks, index)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (28, 2048) + inhomogeneous part.

vectorstore.as_retriever()

- 벡터 저장소를 기반으로 VectorStoreRetriever 객체 생성
- VectorStoreRetriever: 벡터 저장소 기반의 검색기 객체

search_type: 검색 유형("similarity", "mmr", "similarity_score_threshold")
- similarity: 유사도 기반 검색(기본값)
- mmr: Maximal Marginal Relevance 검색
- similarity_score_threshold: 임계값 기반 유사도 검색

검색 매개변수 커스터마이징
- k: 반환할 문서 수 (기본값: 4)
- score_threshold: 유사도 점수 임계값
- fetch_k: MMR 알고리즘에 전달할 문서 수 (기본값: 20)
- lambda_mult: MMR 다양성 조절 파라미터 (기본값: 0.5)
- filter:  문서 메타데이터 기반 필터링

- MMR 검색 시 fetch_k를 높이고, lambda_mult를 조절해 다양성과 관련성의 균형을 맞출 수 있음

In [7]:
# 벡터스토어 저장

vectorstore.save_local(folder_path="data/spiderman_vs")

In [9]:
# 벡터스토어 불러오기

vectorstore = FAISS.load_local(folder_path="data/spiderman_vs", embeddings=embeddings, allow_dangerous_deserialization=True)

In [37]:
# 이 부분 나중에 최적화 실험

retriever = vectorstore.as_retriever()

## RAG를 수행하는 부분

In [14]:
# RAG prompt

prompt = hub.pull("rlm/rag-prompt")

c:\Users\rinap\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [15]:
# RAG에 사용할 모델 정의

rag_llm = ChatOpenAI(
    model_name = "gpt-4-1106-preview",
    temperature=0
)

In [38]:
# QA 체인 정의

qa_chain = RetrievalQA.from_chain_type(
    llm=rag_llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    verbose=True,
    return_source_documents=True
)

In [39]:
start_time = time.time()
result = qa_chain({"query": "얼티밋 유니버스가 뭐야?"})
end_time = time.time()
response_time = end_time - start_time
print(result["result"])
print(f"검색 총 소요 시간: {response_time:.2f}")



> Entering new RetrievalQA chain...

> Finished chain.
얼티밋 유니버스는 메인 유니버스를 기반으로 하되 현실성을 더하고 차별화를 위해 설정을 차용한 세계관입니다. 어메이징 스파이더맨 실사영화 시리즈는 코믹스의 얼티밋 유니버스를 참고했으며, 얼티밋 유니버스는 메인 유니버스보다 현실적인 설정을 바탕으로 리부트한 세계관입니다.
검색 총 소요 시간: 7.24


c:\Users\rinap\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


## langchain agent가 사용할 tool 정의

In [12]:
tools = [
    Tool(
        name="doc_search_tool",
        func=qa_chain,
        description=(
            "This tool is used to retrieve information from the knowledge base"
        )
    )
]

## Agent 정의

In [13]:
system_message = """
    You will act as 신짱구 character and answer the user's questions and interact with the user who is fan of 신짱구.

    You can refer to the following information about 신짱구
    - Characteristic: 짱구는 못말려의 주인공. 신형만, 봉미선 부부의 아들이며, 신짱아의 오빠이다.
    - Personality: 트러블 메이커, 마이 웨이, 호기심이 많음, 장난꾸러기, 사고뭉치, 게으름, 귀찮음, 쑥스럼을 많이 탐.

    You can refer to the following texts to mimic 신짱구 tone and style:
    Here is some text: '안녕하세요, 외국인 누나들.'
    Here is a rewrite of the text, which is 신짱구's manner: '어? 외국인 누나들! 헬로, 헤로헤로헤롱. 이야, 난 신짱구, 다섯 살 아이 러브 유, 알 맛있으유.'

    Here is some text: '전 신짱구예요.'
    Here is a rewrite of the text, which is 신짱구's manner: '전 신짱구예요. 우리 동네에선 쪼끔 유명해요. 떡잎유치원 해바라기반에 다니고 있고 부끄러움을 많이 탄답니다.' 

    You should follow the guidlines below:
    - You must act like a 신짱구 character. Use a first-person perspective. Do not say "전우치는~ "
    - You must answer in Korean
    - You must follow the 신짱구 style naturally.
    - Like the text provided, change the sentence in 신짱구 style.
    - You must refer to the source of documents provided to answer about 신짱구.
    - If the answer isn't available within in the context, state the fact.
    - Otherwise, answer to your best capability, referring to source of documents provided.
    - Answer what you found in the document in 신짱구 style.
    - Limit responses to three or four sentences for clarity and conciseness.
    - Don't use honorifics. Use informal language.
"""

In [14]:
# agent에 사용할 모델 정의

# gpt4: gpt-4
# 스파이더맨 finetuning: ft:gpt-4o-2024-08-06:personal:spiderman-ft-1:A6ZT99j0
# 전우치 finetuning: ft:gpt-4o-2024-08-06:personal:juc-1:AFWso3PG
# 짱구 finetuning: ft:gpt-4o-2024-08-06:personal:szg-1:AFiwN0J2

llm = ChatOpenAI(
    model_name = "ft:gpt-4o-2024-08-06:personal:szg-1:AFiwN0J2",
    temperature=1
)

In [15]:
# agent에 사용할 메모리 정의 (나중에 실험)

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="input",
    return_messages=True,
    output_key="output",
)

In [ ]:
# agent 정의

agent = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    memory=memory,
    return_source_documents=True,
    return_intermediated_steps=True,
    agent_kwargs={"system_message": system_message},
    handle_parsing_errors=True
)

## Agent 출력 테스트

In [ ]:
import time


query = "엉덩이로 뭘 할 수 있어?"

start_time = time.time()

result = agent.run(query)

end_time = time.time()


print(result)
response_time = end_time - start_time

print(f"응답 시간: {response_time:.2f}초")

## G-eval 평가 코드

In [ ]:
evaluation_prompt_template = """
    You will act as an evaluator of an agent's response to a given query.
    The response will be evaluated on three criteria:
    1. **Tone Similarity to Peter Parker (Spider-man)**: How much does the tone of the response resemble the way Peter Parker would speak? (Score: 1-10)
    You can refer to the follwoing lines: '무슨 신, 반짝이 신?', '아니, 널 지켜 주려고 그랬던 거야.', '걱정하지 마, 괜찮을 거야.', '비상사다리 타고. 별거 아니던걸, 뭐.' 
    2. **Politeness**: Is the response polite, free of offensive, violent, or hateful language? (Score: 1-10)
    3. **Engagement/Interestingness*: How engaging and intersting is the response? (Score: 1-10)

    The agent's response to the query is as follows:
    ---
    {response}
    ---

    Please rate the response on each criterion with a score from 1 to 10, and provide a brief explanation for each score in Korean.

    Return the results in this format:
    Tone Similarity: X/10 (Explanation)
    Politeness: Y/10 (Explanation)
    Engagement: Z/10 (Explanation)
"""

evaluation_prompt = PromptTemplate(
    input_variables=["response"],
    template=evaluation_prompt_template
)

evaluation_chain = LLMChain(llm=rag_llm, prompt=evaluation_prompt)

In [ ]:
evaluation_result = evaluation_chain(result["output"])
evaluation_result['text']